In [2]:
import os 
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import random

from datetime import timedelta, datetime

In [ ]:
# DEFINITION OF THE DOMAIN OF INTEREST 

lat_bnd = [-1.2, 0.94]
lon_bnd = [-2.84, 0.02]

# 1. Generating High-Resolution Data 

**DATA :** Sourced from _'data/HadGEM_driven_COSMO'_

**OBJECTIVE :** In this section, independent monthly hourly precipitation maxima will be extracted. This corresponds to the highest hourly precipitation recorded during a summer month (June, July, August). To ensure event independence, all maxima must be spaced at least five days apart. This will be done for the two 11-year periods : 
- _Present_ : between 1999 and 2009.
- _Future_ : between 2079 and 2089. 

**PROCESS :** This is done in three phases:

- **Extraction of initial Maximum** -- Generate the maximum precipitation per month along with the associated timestamp.
- **Conflict Resolution** -- Ensure that all maxima are spaced by at least 5 days. If not:
    - Retain the event with the highest precipitation value.
    - Define upper and lower time constraints for the discarded event.
- **Adjusted Maximum Calculation** -- Recalculate the maximum for cases that need to be adjusted.

## 1.1. Extraction of initial Maximum

**OUTPUT :** 
- The data is stored in  _'repos/Downscaling_CM/data/data_high_resolution'_ 
- Format: _data2_summer_present_HR_v0.csv_.

In [11]:
PERIOD = 'Present/'

if PERIOD == 'Present/': year_min, year_max = 1999, 2010
elif PERIOD == 'Future/' : year_min, year_max = 1999, 2010
else : print('ERROR')

In [ ]:
def get_max_month_HR (name, year, period = 'Present/'):

    '''
    Input: Beginning of the filename for a specific month and year + target year + period ('Present' or 'Future')  
    Output: Dataset containing, for each location in the study domain (rlat / rlon), the maximum hourly precipitation, along with the corresponding year and month.  
    '''

    os.chdir("/data/HadGEM_driven_COSMO/"+period+str(year))
    fichiers_month = [fichier for fichier in os.listdir() if fichier.startswith(name)]
    liste_data = []

    for ind in range(len(fichiers_month)) :
        
        data=xr.open_dataset(fichiers_month[ind])
        
        data = data.sel(rlon = slice(*lon_bnd), rlat = slice(*lat_bnd))
        vars_to_keep = ['TOT_PR', 'rlat', 'rlon', 'time']  
        data = data.drop_vars([var for var in data.variables if var not in vars_to_keep])
        data = data.to_dataframe()
        data= data.reset_index()

        data.rlat = round(data.rlat, 2)
        data.rlon = round(data.rlon, 2)

        liste_data.append(data)

    liste_data = pd.concat(liste_data, ignore_index = True)

    ind = liste_data.groupby(['rlat', 'rlon'])['TOT_PR'].idxmax()
    liste_data = liste_data.loc[ind]

    liste_data['year'] = year
    liste_data['month'] = name[8:10]
    liste_data.reset_index(drop=True, inplace=True)

    return liste_data

liste_data = [get_max_month_HR(f"lffd{year}{mois}", year, PERIOD) for year in range(year_min, year_max) for mois in ['06', '07', '08']]
liste_data = pd.concat(liste_data, ignore_index = True)

os.chdir("/data/data_high_resolution")
liste_data.to_csv('data2_summer_present_HR_v0.csv', index = False)

## 1.2. Conflict Resolution

To ensure the independence of events, we will check whether each event is spaced at least 5 days apart. If not:  
- The highest precipitation value is retained.  
- Lower and upper time constraint values are recorded.  

The values are then organized into three datasets:  
1) **data_clean** → Events without conflicts.  
2) **data_conflict** → Events that require recalculation.  
3) **data_corrected** → Corrected data.  


In [ ]:
# If not in memory 
os.chdir('/data/data_high_resolution')
liste_data = pd.read_csv('data2_summer_present_HR_v0.csv')

In [ ]:
def detect_conflit (liste_data):

    liste_data['time'] = pd.to_datetime(liste_data['time'])
    liste_data['to_change'] = 0 # (0 if there is no conflit, 1 otherwise)
    liste_data['constraint_begin'] = None 
    liste_data['constraint_end'] = None

    for (rlat, rlon, year), df in tqdm.tqdm(liste_data.groupby(['rlat', 'rlon', 'year'], group_keys=False)):
        df = df.sort_values(by='time').reset_index()

        for i in range(len(df) - 1):
            time_i = df.loc[i, 'time']
            time_j = df.loc[i + 1, 'time']

            if abs((time_j - time_i).days) < 5:

                if df.loc[i, 'TOT_PR'] < df.loc[i + 1, 'TOT_PR']:
                    index_to_change = df.loc[i, 'index']
                    time_to_change = df.loc[i, 'time']
                    other_time = df.loc[i + 1, 'time']
                else:
                    other_time = df.loc[i, 'time']
                    index_to_change = df.loc[i + 1, 'index']
                    time_to_change = df.loc[i + 1, 'time']

                liste_data.loc[index_to_change, 'to_change'] = 1 

                month = time_to_change.month

                if month == 6: 
                    liste_data.loc[index_to_change, 'constraint_begin'] = f"{time_to_change.year}-05-31"
                    liste_data.loc[index_to_change, 'constraint_end'] = other_time - pd.Timedelta(days=5)

                elif month == 8:
                    liste_data.loc[index_to_change, 'constraint_begin'] = other_time + pd.Timedelta(days=5)
                    liste_data.loc[index_to_change, 'constraint_end'] = f"{time_to_change.year}-09-01"

                elif month == 7:  
                    june_max = df[df['time'].dt.month == 6]['time'].max()
                    august_max = df[df['time'].dt.month == 8]['time'].min()

                    liste_data.loc[index_to_change, 'constraint_begin'] = june_max + pd.Timedelta(days=5)
                    liste_data.loc[index_to_change, 'constraint_end'] = august_max - pd.Timedelta(days=5)

    return liste_data

liste_data = detect_conflit(liste_data)
data_conflict = liste_data.loc[liste_data.to_change == 1].copy()
data_clean = liste_data.loc[liste_data.to_change == 0].copy()
if len(liste_data) != len(data_clean) + len(data_conflict) : print('-ERROR-')
print('There is ', len(data_conflict), ' conflicts (', np.round(len(data_conflict)/len(liste_data)*100), '%).')

In [ ]:
# Example : analyse d'un conflit
indice = random.choice(data_conflict.index)
year = data_conflict.year[indice]
rlat = data_conflict.rlat[indice]
rlon = data_conflict.rlon[indice]

df_local = liste_data.loc[(liste_data.year == year) &
                          (liste_data.rlat == rlat) &
                          (liste_data.rlon == rlon)]
df_local

## 1.3. Adjusted Maximum Calculation

**OUTPUT :** 
- The data is stored in  _'/data/data_high_resolution'_ 
- Format: _data2_summer_present_HR_v1.csv_.

In [ ]:
data_corrected = []
PERIOD = 'Present/'

In [ ]:
def get_max_month_HR_with_constraints (name, year, period, constraint_begin, constraint_end, lat_bnd, lon_bnd):

    os.chdir("/data/HadGEM_driven_COSMO/"+period+str(year))
    fichiers_month = [fichier for fichier in os.listdir() if fichier.startswith(name)]
    liste_data = []

    for ind in range(len(fichiers_month)) :
        
        data=xr.open_dataset(fichiers_month[ind])
        
        data = data.sel(rlon = slice(*lon_bnd), rlat = slice(*lat_bnd))
        vars_to_keep = ["TOT_PR", 'rlat', 'rlon', 'time']  
        data = data.drop_vars([var for var in data.variables if var not in vars_to_keep])
        data = data.to_dataframe()
        data= data.reset_index()
        data = data.loc[(data.time >= constraint_begin) & (data.time <= constraint_end)]
        
        data.rlat = round(data.rlat, 2)
        data.rlon = round(data.rlon, 2)

        liste_data.append(data)

    liste_data = pd.concat(liste_data, ignore_index = True)

    ind = liste_data.groupby(['rlat', 'rlon'])['TOT_PR'].idxmax()
    liste_data = liste_data.loc[ind]

    liste_data['year'] = year
    liste_data['month'] = name[8:10]
    liste_data.reset_index(drop=True, inplace=True)

    return liste_data

In [ ]:
data_conflict[['constraint_end', 'constraint_begin']] = data_conflict[['constraint_end', 'constraint_begin']].apply(pd.to_datetime)

for year in range(1999, 2010):
    for month in [6, 7, 8]:

        # Initialisation 
        mois = f"{month:02d}"
        name = f"lffd{year}{mois}"
        
        df_local = data_conflict.loc[(data_conflict.year == year) & ((data_conflict.month == month))].copy()

        if month == 7:
            month_conditions = [
                (df_local.constraint_end.dt.month == 7) & (df_local.constraint_begin.dt.month == 7),
                (df_local.constraint_end.dt.month == 7) & (df_local.constraint_begin.dt.month == 6),
                (df_local.constraint_end.dt.month == 8) & (df_local.constraint_begin.dt.month == 7)
            ]
        else:
            month_conditions = [df_local.index]  # Condition unique pour juin et août
        
        ### 
        for condition in month_conditions:
            df_sub = df_local.loc[condition]

            ind_prev = -1
            
            # Check groupé 
            while len(df_sub) > 0 and len(df_sub) != ind_prev:
                ind_prev = len(df_sub)

                lat_bnd = [df_sub.rlat.min(), df_sub.rlat.max()]
                lon_bnd = [df_sub.rlon.min(), df_sub.rlon.max()]

                df_result = get_max_month_HR_with_constraints(
                    name, year, PERIOD, df_sub.constraint_begin.min(),
                    df_sub.constraint_end.max(), lat_bnd, lon_bnd)
                df_sub.drop(columns=['TOT_PR', 'time'], inplace = True)

                for df in [df_sub, df_result]:
                    df[['rlat', 'rlon']] = df[['rlat', 'rlon']].round(2).astype('float')

                df_merge = pd.merge(df_sub, df_result[['rlat', 'rlon', 'TOT_PR', 'time']], on=['rlat', 'rlon'], how='left')

                # Update 
                data_corrected.append(df_merge.loc[
                    (df_merge.constraint_begin <= df_merge.time) &
                    (df_merge.constraint_end >= df_merge.time)])
                
                df_sub = df_merge.loc[
                    (df_merge.constraint_begin > df_merge.time) |
                    (df_merge.constraint_end < df_merge.time)]
                

            # Check individuel 
            for _, df_remain in df_sub.iterrows():
                df_remain = df_remain.to_frame().T 

                lat_bnd = [df_remain.rlat.min(), df_remain.rlat.max()]
                lon_bnd = [df_remain.rlon.min(), df_remain.rlon.max()]

                df_result = get_max_month_HR_with_constraints(
                    name, year, PERIOD, df_remain.constraint_begin.min(),
                    df_remain.constraint_end.max(), lat_bnd, lon_bnd
                )

                df_remain[['rlat', 'rlon']] = df_remain[['rlat', 'rlon']].round(2).astype('float64')
                df_result[['rlat', 'rlon']] = df_result[['rlat', 'rlon']].round(2).astype('float64')

                df_merge = df_remain.drop(columns=['TOT_PR', 'time']).merge(
                    df_result[['rlat', 'rlon', 'TOT_PR', 'time']], on=['rlat', 'rlon'], how='left'
                )

                data_corrected.append(df_merge)


In [ ]:
df_part1 = liste_data.loc[liste_data.to_change == 1].copy()
df_part2 = pd.concat(data_corrected)
df_result = pd.concat([df_part1, df_part2])
df.result.to_csv('data2_summer_present_HR_v1.csv')

# 2. Generating Low-Resolution Data 

**DATA :** Sourced from _'downscaling/HadGEM_driven_COSMO'_

**OBJECTIVE :** In this section, low-resolution data will be generated from the same dataset to establish an ideal framework. Then independent monthly hourly precipitation maxima will be extracted. This corresponds to the highest hourly precipitation recorded during a summer month (June, July, August). To ensure event independence, all maxima must be spaced at least five days apart. This will be done for the two 11-year periods : 
- _Present_ : between 1999 and 2009.
- _Future_ : between 2079 and 2089. 

**PROCESS :** This is done in five phases:

- **Create the grille** -- 
- **Data Blurring** -- Apply mean pooling to the data to reach the desired resolution. 
- **Extraction of initial Maximum** -- Generate the maximum precipitation per month along with the associated timestamp.
- **Conflict Resolution** -- Ensure that all maxima are spaced by at least 5 days. If not:
    - Retain the event with the highest precipitation value.
    - Define upper and lower time constraints for the discarded event.
- **Adjusted Maximum Calculation** -- Recalculate the maximum for cases that need to be adjusted.

In [ ]:
RESOLUTION = 24

In [ ]:
PERIOD = 'Present/'

if PERIOD == 'Present/':
    year_min, year_max = 1999, 2010
    period_text = 'present'
elif PERIOD == 'Future/' :
    year_min, year_max = 1999, 2010
    period_text = 'future'
else : print('ERROR')

## 2.1. Create the grille

In [ ]:
resolution = int(RESOLUTION / 2)

# ---------------------------------------------------------------------------

# Intersection points at 2 km resolution
liste_lat_2 = np.arange(lat_bnd[0], lat_bnd[1]+0.01, 0.02)
liste_lon_2 = np.arange(lon_bnd[0], lon_bnd[1]+0.01, 0.02)

# Intersection points at 12 km resolution
liste_lat_12 = liste_lat_2[::resolution]
liste_lon_12 = liste_lon_2[::resolution]

# Ensure coverage by appending an extra point
liste_lat_12 = np.append(liste_lat_12, np.max(liste_lat_2) + 0.01)
liste_lon_12 = np.append(liste_lon_12, np.max(liste_lon_2) + 0.01)

# ---------------------------------------------------------------------------

# Create the coordinate DataFrame
lat_indices = np.arange(len(liste_lat_2))
lon_indices = np.arange(len(liste_lon_2))
lat_grid, lon_grid = np.meshgrid(lat_indices, lon_indices)

dico_coord = pd.DataFrame({'rlat': np.array(liste_lat_2)[lat_grid.ravel()], 
                            'rlon': np.array(liste_lon_2)[lon_grid.ravel()]})

# Assign a block number for each coordinate
dico_coord['block'] = 'X'

ind_block = 0
for lat in liste_lat_12:
    for lon in liste_lon_12:
        mask = ((dico_coord.rlat < lat) & (dico_coord.rlon < lon) & (dico_coord.block == 'X'))
        if mask.any():
            dico_coord.loc[mask, 'block'] = ind_block
            ind_block += 1

# --------------------------------------------------------------------------- 

# Summarize data per block
data_block = dico_coord.groupby('block').agg({'rlat': 'mean', 'rlon': 'mean'}).reset_index()

# ---------------------------------------------------------------------------

# Find the nearest blocks for each position
def find_min (lat, lon, df):
    
    distance = np.sqrt((df.rlat - lat)**2 + (df.rlon - lon)**2)
    min_distance_index = distance.idxmin()
    
    block = df.loc[min_distance_index, 'block']

    df = df.loc[df['block'] != block]
    
    return block, df

# Initialize columns for neighbors
dico_coord[['block1', 'block2']] = 'X', 'X'

for ind in tqdm.tqdm(range(len(dico_coord))):
    lon = dico_coord.rlon[ind]
    lat = dico_coord.rlat[ind]
    block = dico_coord.block[ind]

    df = data_block.loc[data_block.block != block].copy()

    block2, df = find_min(lat, lon, df)

    dico_coord.loc[(dico_coord.rlat == lat) & (dico_coord.rlon == lon), ['block1', 'block2']] = [block, block2]
   
dico_coord.rlat = round(dico_coord.rlat, 2)

# ---------------------------------------------------------------------------

# Save the DataFrame to a CSV file
dico_coord.to_csv('grille2_'+str(RESOLUTION)+'.csv', index=False)

# Extracting simplified grid for further use
grille = dico_coord[['rlat', 'rlon', 'block']].copy()


In [ ]:
grille = grille.loc[grille.rlon >-0.5]
grille = grille.loc[grille.rlat >0.5]

# Assign a unique color for each block
groups = grille.groupby('block')
colors = plt.cm.get_cmap('tab20', len(groups))

plt.figure(figsize=(10, 8))

for i, (block, group) in enumerate(groups):
    plt.scatter(group.rlon, group.rlat, color=colors(i), label=f'Block {block}', s=100)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter plot of grid points colored by block')
plt.grid(True)
plt.tight_layout()
plt.show()

## 2.1. & 2.2. Data Blurring & Extraction of initial Maximum 

In [ ]:
def get_max_month_LR (name, year, period = 'Present/'):

    '''
    Input: Beginning of the filename for a specific month and year + target year + period ('Present' or 'Future')  
    Output: Dataset containing, for each block in the study domain (rlat / rlon), the maximum hourly precipitation, along with the corresponding year and month.  
    '''

    os.chdir("/downscaling/HadGEM_driven_COSMO/"+period+str(year))
    fichiers_month = [fichier for fichier in os.listdir() if fichier.startswith(name)]
    liste_data = []

    for ind in range(len(fichiers_month)) :

        data=xr.open_dataset(fichiers_month[ind])
        
        data = data.sel(rlon = slice(*lon_bnd), rlat = slice(*lat_bnd))
        vars_to_keep = ['rlat', 'rlon', 'TOT_PR', 'time']  
        data = data.drop_vars([var for var in data.variables if var not in vars_to_keep])
        data = data.to_dataframe()
        data= data.reset_index()

        data.rlat = round(data.rlat, 2)
        data.rlon = round(data.rlon, 2)
        data = pd.merge(data, grille, on = ['rlat', 'rlon'], how = 'left')
        data_time = data.time[0]
        data = data.groupby(['block'])['TOT_PR'].mean().reset_index()
        data['time'] = data_time

        liste_data.append(data)

    liste_data = pd.concat(liste_data, ignore_index = True)
    liste_data = liste_data.reset_index() 
    
    ind = liste_data.groupby(['block'])['TOT_PR'].idxmax()
    liste_data = liste_data.loc[ind]

    liste_data['year'] = year
    liste_data['month'] = name[8:10]

    return liste_data 

liste_data = [get_max_month_HR(f"lffd{year}{mois}", year, PERIOD) for year in range(year_min, year_max) for mois in ['06', '07', '08']]
liste_data = pd.concat(liste_data, ignore_index = True)

os.chdir("/data/data_high_resolution")
liste_data.to_csv('data2_summer_'+period_text'_HR_v0.csv', index = False)

## 2.3. Adjusted Maximum Calculation

In [ ]:
def detect_conflit (liste_data):

    liste_data['time'] = pd.to_datetime(liste_data['time'])
    liste_data['to_change'] = 0 # (0 if there is no conflit, 1 otherwise)
    liste_data['constraint_begin'] = None 
    liste_data['constraint_end'] = None

    for (block, year), df in tqdm.tqdm(liste_data.groupby(['block', 'year'], group_keys=False)):
        df = df.sort_values(by='time').reset_index()

        for i in range(len(df) - 1):
            time_i = df.loc[i, 'time']
            time_j = df.loc[i + 1, 'time']

            if abs((time_j - time_i).days) < 5:

                if df.loc[i, 'TOT_PR'] < df.loc[i + 1, 'TOT_PR']:
                    index_to_change = df.loc[i, 'index']
                    time_to_change = df.loc[i, 'time']
                    other_time = df.loc[i + 1, 'time']
                else:
                    other_time = df.loc[i, 'time']
                    index_to_change = df.loc[i + 1, 'index']
                    time_to_change = df.loc[i + 1, 'time']

                liste_data.loc[index_to_change, 'to_change'] = 1 

                month = time_to_change.month

                if month == 6: 
                    liste_data.loc[index_to_change, 'constraint_begin'] = f"{time_to_change.year}-05-31"
                    liste_data.loc[index_to_change, 'constraint_end'] = other_time - pd.Timedelta(days=5)

                elif month == 8:
                    liste_data.loc[index_to_change, 'constraint_begin'] = other_time + pd.Timedelta(days=5)
                    liste_data.loc[index_to_change, 'constraint_end'] = f"{time_to_change.year}-09-01"

                elif month == 7:  
                    june_max = df[df['time'].dt.month == 6]['time'].max()
                    august_max = df[df['time'].dt.month == 8]['time'].min()

                    liste_data.loc[index_to_change, 'constraint_begin'] = june_max + pd.Timedelta(days=5)
                    liste_data.loc[index_to_change, 'constraint_end'] = august_max - pd.Timedelta(days=5)

    return liste_data

liste_data = detect_conflit(liste_data)
data_conflict = liste_data.loc[liste_data.to_change == 1].copy()
data_clean = liste_data.loc[liste_data.to_change == 0].copy()
if len(liste_data) != len(data_clean) + len(data_conflict) : print('-ERROR-')
print('There is ', len(data_conflict), ' conflicts (', np.round(len(data_conflict)/len(liste_data)*100), '%).')

In [ ]:
def get_max_month_LR_with_constraints (name, year, period, constraint_begin, constraint_end, block):

    os.chdir("/downscaling/HadGEM_driven_COSMO/"+period +str(year))
    fichiers_month = [fichier for fichier in os.listdir() if fichier.startswith(name)]
    liste_data = []

    for ind in range(len(fichiers_month)) :

        data=xr.open_dataset(fichiers_month[ind])
        
        data = data.sel(rlon = slice(*lon_bnd), rlat = slice(*lat_bnd))
        vars_to_keep = ['rlat', 'rlon', 'TOT_PR', 'time']  
        data = data.drop_vars([var for var in data.variables if var not in vars_to_keep])
        data = data.to_dataframe()
        data= data.reset_index()
        data = data.loc[(data.time >= constraint_begin) & (data.time <= constraint_end)]

        data.rlat = round(data.rlat, 2)
        data.rlon = round(data.rlon, 2)

        data = pd.merge(data, grille, on = ['rlat', 'rlon'], how = 'left')
        data = data.loc[data['block'].isin(block)]

        data_time = data.time[0]
        data = data.groupby(['block'])['TOT_PR'].mean().reset_index()
        data['time'] = data_time
        liste_data.append(data)

    liste_data = pd.concat(liste_data, ignore_index = True)
    liste_data = liste_data.reset_index() 
    
    ind = liste_data.groupby(['block'])['TOT_PR'].idxmax()
    liste_data = liste_data.loc[ind]

    liste_data['year'] = year
    liste_data['month'] = name[8:10]
    liste_data.reset_index(drop=True, inplace=True)

    return liste_data 

In [ ]:
df_local0 = liste_data.loc[liste_data.to_change == 1].copy()
df_local0['constraint_end'] = pd.to_datetime(df_local0['constraint_end'])
df_local0['constraint_begin'] = pd.to_datetime(df_local0['constraint_begin'])
df_local0.reset_index(inplace = True)

liste_data_new = []
pb = []

for index in tqdm.tqdm(df_local0.index) :
       df_local = df_local0.loc[df_local0.index == index]
       year = df_local0.loc[index, 'year']


       mois = '0'+str(df_local0.loc[index, 'month'])[0]
       name = 'lffd'+str(int(year)) + mois
      
       print(len(df_local))


       df_result = get_max_month_LR_with_constraints (name, year, min(df_local.constraint_begin),
                                                       max(df_local.constraint_end),
                                                       list(df_local.block.unique()))
       if not isinstance(df_result, pd.DataFrame): pb.append(index)
       else :
           df_local = df_local.drop(columns = ['TOT_PR', 'time'])


           df_merge = pd.merge(df_local, df_result[['block', 'TOT_PR', 'time']], on = ['block'], how = 'left')
           print('Warning :' , np.sum(df_merge.TOT_PR.isna()))


           liste_data_new.append(df_merge.loc[(df_merge['time'] <= df_merge['constraint_end']) & (df_merge['time'] >= df_merge['constraint_begin'])])
           df_local = df_merge.loc[(df_merge['time'] > df_merge['constraint_end']) | (df_merge['time'] < df_merge['constraint_begin'])]
           if len(df_local)>0:
               pb.append(index)



In [ ]:
df_result = pd.concat([liste_data.loc[liste_data.to_change == 0],
          pd.concat(liste_data_new, ignore_index = True)],
          ignore_index = True)


os.chdir("/data/data_low_resolution")
d.to_csv('data2_summer_future_LR24_v1.csv', index = False)